In [1]:
import os
owd = os.getcwd()

In [2]:
os.chdir(owd)
import numpy as np
import re
import pandas as pd
import ipywidgets as wdg
from plotly_default import go
from TecplotTools import read_tecplot, data_cats, plot_var_range, plot_bounds

In [3]:
# Import the plot categories
os.chdir(owd)
plot_cats = pd.read_csv('./plot_categories.csv', keep_default_na=False)

# Get list of tec files (note that this changes the cwd)
catList, max_time = data_cats(r"D:/Peter/Code/CrunchTope/CS_Link")

# Get the vertical scale
with open(r'CSColumnShortARSR.in') as f:
   all_lines = f.read()
   distance_units = re.search('distance_units +(\w+)', all_lines).group(1)
   x_res = re.search('xzones +(\d+) (\d+)', all_lines).group(2)

# filter categories
# plot_cats = plot_cats[(plot_cats['file_name'].isin(catList)) & (plot_cats.ignore == False)]
plot_cats.query('file_name in @catList & ignore == False', inplace=True)
plot_cats

,file_name,friendly_name,prefix,suffix,units,starts_log,ignore
0,AqRate,Aqueous reaction rate,Rate of,,mol dm<sup>−3</sup> s<sup>−1</sup>,False,False
1,TotMineral,Component concentration in minerals,Concentration of,in minerals,mol kg<sup>−1</sup>,False,False
2,affinity_term,Monod affinity term,Affinity term for,,,False,False
3,aq_affinity,Monod biomass aqueous affinity term,Affinity term for,,,False,False
4,area,Mineral surface area,Surface area of,,m<sup>2</sup>,False,False
5,conc,Aqueous species concentration,Concentration of,,mol kg<sup>−1</sup>,True,False
6,pH,pH,,,,False,False
7,porosity,Porosity,,,,False,False
8,rate,Mineral precipitation rate,Rate of,precipitation,mol m<sup>−3</sup> s<sup>−1</sup>,False,False
9,saturation,Mineral supersaturation,Ω,,,False,False


In [7]:
from turtle import distance


x_var_type = wdg.Dropdown(value='Component concentration', options=plot_cats['friendly_name'])

def get_filename(x_var_type):
    return str(plot_cats.query(f"friendly_name == '{x_var_type.value}'")['file_name'].iloc[0])

time = wdg.IntSlider(value=1,
    min=1,
    max=max_time,
    step=1)

play = wdg.Play(value=1, min=1, max=max_time, step=1, interval=1)
time_slider = wdg.IntSlider(value=1, min=1, max=max_time, step=1, layout=wdg.Layout(width='50%'))
wdg.jslink((play, 'value'), (time_slider, 'value'))

x_var = wdg.Dropdown(options=(read_tecplot(get_filename(x_var_type), time.value)[1])[3:])

def initialise1D(x_var_type):
    df, column_headers = read_tecplot(x_var_type, 1)
    df.eval('X_scaled = X * @x_res', inplace=True)
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(y=df["X_scaled"], x=np.zeros_like(df["X_scaled"])))
    line = fig.select_traces().__next__()
    fig.update_layout(
        yaxis=dict(
            autorange="reversed", rangemode="tozero", title=f"Depth, <i>z</i> ({distance_units})"
        ),
        hovermode="y",
        width=600,
        height=700,
    )
    return fig, line

def update_x_vars(*args):
    x_vars_list = read_tecplot(get_filename(x_var_type), time.value)[1]
    x_var.options = x_vars_list[3:]

def update_plot(time, x_var_type, x_var):
    plot_info = plot_cats.query(f"friendly_name == '{x_var_type}'")
    
    file_cat = plot_info['file_name'].iloc[0]
    pref_str = plot_info['prefix'].iloc[0]
    suff_str = plot_info['suffix'].iloc[0]
    unit_str = plot_info['units'].iloc[0]
    if unit_str != '':
        unit_str = '(' + unit_str + ')'
    
    df, column_headers = read_tecplot(file_cat, time)
        
    lower, upper =  plot_var_range(max_time, file_cat, x_var)
    lower, upper = plot_bounds(lower, upper)
    
    with fig.batch_update():
        line.x = df[x_var]
        
        fig.update_layout(xaxis_range = [lower, upper],
                          xaxis_title = f'{pref_str} {x_var} {suff_str} {unit_str}')

x_var_type.observe(update_x_vars, 'value')

fig, line = initialise1D('totcon')

wdg.interactive(update_plot, x_var_type=x_var_type, time=time_slider, x_var=x_var, max_time=wdg.fixed(max_time))

# Display widgets
wdg.VBox([
    wdg.HBox([x_var_type, x_var]),
    wdg.HBox([play, time_slider]),
    wdg.HBox([fig])
])